# BRSET Fundus Image Classification with GPT-4o

This notebook classifies fundus images in the BRSET dataset for the presence of 'drusens' using GPT-4o. It compares model predictions across age and sex groups, analyzing potential bias using key evaluation metrics.

## 1. Import Libraries

In [1]:
import pandas as pd
import requests
import base64
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import matplotlib.pyplot as plt

## 2. API Functions

In [2]:
def encode_image(image_path):
    """
    Encode an image file to base64 string
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def classify_image(api_key, image_path, question):
    """
    Make an API call to GPT-4o API for image classification
    """
    api_endpoint = "https://api.openai.com/v1/chat/completions"
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 300
    }

    response = requests.post(api_endpoint, headers=headers, json=payload)
    return int(response.json()['choices'][0]['message']['content'].strip())


## 3. Load Dataset and Prepare Experiments

In [3]:
# Define paths to the datasets
age_experiment_path = 'drusens_only/experiment1_ages/dataset/labels1.csv'
sex_experiment_path = 'drusens_only/experiment2_genders/dataset/labels2.csv'

# Load the datasets
age_experiment = pd.read_csv(age_experiment_path)
sex_experiment = pd.read_csv(sex_experiment_path)

# Define the OpenAI API key and the classification question
api_key = 'API_KEY'
question = "You are an AI ophthalmology assistant analyzing a fundus image to check exclusively for the presence of Drusen. There may be other conditions present, but they are not relevant to this task. If Drusen is present, respond with the digit 1. If Drusen is not present, respond with the digit 0. Only respond with the single digit 1 or 0; provide no explanations or additional text. This is a strict format requirement."

## 4. Perform Classification and Collect Results

In [4]:
# Function to get predictions for a dataset
def get_predictions(df, demographic_column, experiment_images_path):
    predictions = []
    for _, row in df.iterrows():
        image_path = f"{experiment_images_path}/{row['image_id']}.jpg"  # Correct image path based on experiment
        actual = row['drusens']
        predicted = classify_image(api_key, image_path, question)
        
        predictions.append({
            'image_id': row['image_id'],
            'patient_id': row['patient_id'],
            demographic_column: row[demographic_column],
            'drusens_actual': actual,
            'drusens_predicted': predicted
        })
    return pd.DataFrame(predictions)

# Define image paths for each experiment
age_experiment_images_path = 'drusens_only/experiment1_ages/dataset/images'
sex_experiment_images_path = 'drusens_only/experiment2_genders/dataset/images'

In [5]:
# Get predictions for age experiment
age_results = get_predictions(age_experiment, 'age_group', age_experiment_images_path)

In [6]:
# Get predictions for gender experiment
sex_results = get_predictions(sex_experiment, 'patient_sex', sex_experiment_images_path)

## 5. Save Results

In [8]:
age_results.to_csv('drusens_only/experiment1_ages/results/age_experiment_results_gpt4o.csv', index=False)
sex_results.to_csv('drusens_only/experiment2_genders/results/sex_experiment_results_gpt4o.csv', index=False)